## Домашнее задание 2. Основные операторы PostgreSQL

#### 1. Вывести все уникальные бренды, у которых есть хотя бы один продукт со стандартной стоимостью выше 1500 долларов, и суммарными продажами не менее 1000 единиц.

In [ ]:
select
	distinct brand
from product
where 1=1
	and standard_cost > 1500
	and product_id in (select
						          product_id
                      from order_items
                      group by product_id
                      having sum(quantity) > 1000
					          	)
;

#### 2. Для каждого дня в диапазоне с 2017-04-01 по 2017-04-09 включительно вывести количество подтвержденных онлайн-заказов и количество уникальных клиентов, совершивших эти заказы.

In [ ]:
select
	--order_date,
	count(distinct order_id) as "Количество подтвержденных онлайн-заказов"
 	,count(distinct customer_id) "Количество уникальных клиентов"
from orders o
where
	order_date between '2017-04-01' and '2017-04-09' --date('2017-04-01') and date('2017-04-09')
	and order_status = 'Approved'
	and online_order = true
group by order_date
order by order_date


#### 3. Вывести профессии клиентов:
  из сферы IT, чья профессия начинается с Senior;
  из сферы Financial Services, чья профессия начинается с Lead.
Для обеих групп учитывать только клиентов старше 35 лет. Объединить выборки с помощью UNION ALL.



In [ ]:

select
	job_title
	--,job_industry_category
	--,dob
	--,EXTRACT(YEAR FROM AGE('2025-11-17', dob))
from dz2.customer
where job_industry_category = 'IT'
	and job_title like 'Senior%'
	and EXTRACT(YEAR FROM AGE('2025-11-17', dob)) > 35
union all
select
	job_title
	--,job_industry_category
	--,dob
	--,EXTRACT(YEAR FROM AGE('2025-11-17', dob))
from dz2.customer
where job_industry_category = 'Financial Services'
	and job_title like 'Lead%' --таких должностей нет в Financial Services
	and EXTRACT(YEAR FROM AGE('2025-11-17', dob)) > 35
;


#### 4. Вывести бренды, которые были куплены клиентами из сферы Financial Services, но не были куплены клиентами из сферы IT.

In [ ]:

with job_order_brand as (
select-- *
	o.order_id
	,o.customer_id
	,c.job_industry_category
	,i.product_id
	,p.brand
from orders o
left join customer c on c.customer_id = o.customer_id
left join order_items i on o.order_id = i.order_id
join product p on i.product_id = p.product_id
--where job_industry_category != 'IT'
)

select distinct brand
from job_order_brand
where 1=1
	and job_industry_category = 'Financial Services'
	and brand not in (select brand from job_order_brand where job_industry_category != 'IT')
;


#### 5. Вывести 10 клиентов (ID, имя, фамилия), которые совершили наибольшее количество онлайн-заказов (в штуках) брендов Giant Bicycles, Norco Bicycles, Trek Bicycles, при условии, что они активны и имеют оценку имущества (property_valuation) выше среднего среди клиентов из того же штата.


In [ ]:
with avg_property_valuation as (
select state, avg(property_valuation) as mean_value from customer c
group by state
)

select
	c.customer_id,
	c.first_name,
	c.last_name
	--concat(c.first_name, concat(' ', c.last_name)) as Name,
	--count(distinct ord.order_id) as amount_of_orders
from customer c
inner join avg_property_valuation avg on c.state = avg.state
left join orders ord on ord.customer_id = c.customer_id
left join order_items oi on oi.order_id = ord.order_id
left join product p on p.product_id = oi.product_id
where 1=1
	and online_order = true
	and deceased_indicator = 'Y' --только 2 активных клиента
	and property_valuation > mean_value
	and brand in ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
group by 	c.customer_id, c.first_name, c.last_name
order by count(ord.order_id) desc
limit 10
;

#### 6. Вывести всех клиентов (ID, имя, фамилия), у которых нет подтвержденных онлайн-заказов за последний год, но при этом они владеют автомобилем и их сегмент благосостояния не Mass Customer.


In [ ]:

select
	c.customer_id,
	c.first_name,
	c.last_name
from customer c
join orders ord on ord.customer_id = c.customer_id
where 1=1
  and online_order = true
	and order_status != 'Approved'
	and order_date::DATE > (SELECT MAX(order_date::DATE) - INTERVAL '1 year' AS date_one_year_ago FROM orders)
	and owns_car = 'Yes'
	and wealth_segment != 'Mass Customer'
order by 1
;


#### 7. Вывести всех клиентов из сферы 'IT' (ID, имя, фамилия), которые купили 2 из 5 продуктов с самой высокой list_price в продуктовой линейке Road.

In [ ]:
select
	c.customer_id,
	first_name,
	last_name
from customer c
left join orders ord on ord.customer_id = c.customer_id
left join order_items oi on oi.order_id = ord.order_id
left join product p on p.product_id = oi.product_id
where 1=1
	and job_industry_category = 'IT'
	and p.product_id in (select product_id from product order by list_price desc limit 5)
	and brand like '%Road%' -- нет такой линейки
	--and brand = 'Trek Bicycles' --как пример другой бренд
group by c.customer_id, first_name, last_name
having count(distinct ord.order_id) = 2
order by 1
;


#### 8. Вывести клиентов (ID, имя, фамилия, сфера деятельности) из сфер IT или Health, которые совершили не менее 3 подтвержденных заказов в период 2017-01-01 по 2017-03-01, и при этом их общий доход от этих заказов превышает 10 000 долларов. Разделить вывод на две группы (IT и Health) с помощью UNION.

In [ ]:


select
	c.customer_id,
	c.first_name,
	c.last_name,
	job_industry_category
	--,count(distinct o.order_id),
	--sum(item_list_price_at_sale * quantity) as count_all
	--* --distinct p.product_id, p.brand
from orders o
left join customer c on c.customer_id = o.customer_id
left join order_items i on o.order_id = i.order_id
join product p on i.product_id = p.product_id
where 1=1
	and job_industry_category = 'IT'
	and order_status = 'Approved'
	and order_date between '2017-01-01' and '2017-03-01'
group by c.customer_id,
	c.first_name,
	c.last_name, job_industry_category
having count(distinct o.order_id) >= 3
	and sum(item_list_price_at_sale * quantity) > 10000
union all
select
	c.customer_id,
	c.first_name,
	c.last_name,
	job_industry_category
from orders o
left join customer c on c.customer_id = o.customer_id
left join order_items i on o.order_id = i.order_id
join product p on i.product_id = p.product_id
where 1=1
	and job_industry_category = 'Health'
	and order_status = 'Approved'
	and order_date between '2017-01-01' and '2017-03-01'
group by c.customer_id,
	c.first_name,
	c.last_name, job_industry_category
having count(distinct o.order_id) >= 3
	and sum(item_list_price_at_sale * quantity) > 10000
order by customer_id
;